## Описание данных и постановка задачи

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.\
Первая задача как аналитика — проанализировать базу данных.\
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.\
Эти данные помогут сформулировать ценностное предложение для нового продукта.


**Таблица books**\
Содержит данные о книгах:\
book_id — идентификатор книги;\
author_id — идентификатор автора;\
title — название книги;\
num_pages — количество страниц;\
publication_date — дата публикации книги;\
publisher_id — идентификатор издателя.

**Таблица authors**\
Содержит данные об авторах:\
author_id — идентификатор автора;\
author — имя автора.

**Таблица publishers**\
Содержит данные об издательствах:\
publisher_id — идентификатор издательства;\
publisher — название издательства;

**Таблица ratings**\
Содержит данные о пользовательских оценках книг:\
rating_id — идентификатор оценки;\
book_id — идентификатор книги;\
username — имя пользователя, оставившего оценку;\
rating — оценка книги.

**Таблица reviews**\
Содержит данные о пользовательских обзорах на книги:\
review_id — идентификатор обзора;\
book_id — идентификатор книги;\
username — имя пользователя, написавшего обзор;\
text — текст обзора.

Задачи\
Посчитайте, сколько книг вышло после 1 января 2000 года;\
Для каждой книги посчитайте количество обзоров и среднюю оценку;\
Определите издательство, которое выпустило наибольшее число книг толще\
50 страниц — так вы исключите из анализа брошюры;\
Определите автора с самой высокой средней оценкой книг — учитывайте\
только книги с 50 и более оценками;\
Посчитайте среднее количество обзоров от пользователей, которые поставили
больше 50 оценок.

## Импорт библиотек и подключание к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
              'pwd': 'Sdf4$2;d-d30pp', # пароль
              'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
              'port': 6432, # порт подключения
              'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'});

Напишем запрос на вывод общей информации по всем таблицам, чтобы проверить состав  данным

In [2]:
# создаём запрос на вывод из списка табилц
table = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for i in table:
    query= ''' 
    SELECT *
    FROM {}
    --первые 5 строк
    LIMIT 5  '''.format(i)
    # получаем данные на основании запроса
    date = pd.io.sql.read_sql(query, con = engine)
    # выводим на экран 
    display(date)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задачи

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [3]:
query = '''
SELECT COUNT(book_id) AS количество_книг 
FROM books
WHERE publication_date::DATE > '2000-01-01';
'''

In [4]:
data = pd.io.sql.read_sql(query, con = engine)
display(data)

,количество_книг
0,819


###  Для каждой книги посчитайте количество обзоров и среднюю оценку

In [5]:
query ='''
SELECT  
    b.title AS название_книги,
    COUNT(DISTINCT rw.review_id) AS кол_во_обзоров, 
    AVG(rt.rating) AS средняя_оценка
FROM books AS b
    LEFT OUTER JOIN reviews AS rw ON b.book_id = rw.book_id
    LEFT OUTER JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id
ORDER BY средняя_оценка DESC, кол_во_обзоров DESC
'''

In [6]:
data = pd.io.sql.read_sql(query, con = engine)
display(data)

,название_книги,кол_во_обзоров,средняя_оценка
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,Wherever You Go There You Are: Mindfulness Me...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

In [7]:
query = '''
SELECT
    publishers.publisher AS издательство,
    COUNT(book_id) AS количество
FROM books INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY количество DESC
LIMIT 1;
'''


In [8]:
data = pd.io.sql.read_sql(query, con = engine)
display(data)

,издательство,количество
0,Penguin Books,42


###  Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [9]:
query = '''
SELECT
    author AS автор,
    AVG(subq.rating) AS ср_рейтинг
FROM (
    SELECT
        author,
        title,
        AVG(rating) AS rating
    FROM (
        SELECT *
        FROM books
        WHERE book_id IN (
            SELECT book_id
            FROM ratings
            GROUP BY book_id
            HAVING COUNT(rating) >= 50
        ) 
    ) AS rated_books
    INNER JOIN authors ON rated_books.author_id = authors.author_id
    INNER JOIN ratings ON rated_books.book_id = ratings.book_id
    GROUP BY author, title
    ORDER BY author
) AS subq
GROUP BY author
ORDER BY ср_рейтинг DESC
LIMIT 1
'''

In [10]:
data = pd.io.sql.read_sql(query, con = engine)
display(data)

,автор,ср_рейтинг
0,J.K. Rowling/Mary GrandPré,4.283844


### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
# создаём запрос
query = '''
SELECT
    AVG(subq.review_count) AS среднее_кол_во_польз_
FROM (
    SELECT COUNT(review_id) AS review_count
        FROM
            reviews
        WHERE username IN (
            SELECT username
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating) > 50
            )
    GROUP BY username
    ) AS subq;
'''

In [12]:
data = pd.io.sql.read_sql(query, con = engine)
display(data)

,среднее_кол_во_польз_
0,24.333333


## Общий вывод:


Посчитано, сколько книг вышло после 1 января 2000 года - 819 книг, также посчитано для каждой книги  количество обзоров и среднюю оценку\
Определено издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books;\
Определен автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré;\
Посчитано среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24,3